<h1>Vorhersage, ob es bei einem Unfall Schwerverletzte/Tote gibt</h1>

## 1. Vorbereitung des Datensatzes

### 1.1 Generelles Aufbereiten der Datenbasis

In [17]:
import geopandas as gpd
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
crash_data = gpd.read_file('../01_Source/U_VM_A_L_BRW_TL.geojson') 
geo_information = pd.read_csv('../01_Source/geo_information.csv', sep=';')
crash_data = pd.merge(crash_data, geo_information, on ='OBJECTID',  how='inner')
from bs4 import BeautifulSoup
import requests
import numpy as np
import datetime

In [18]:
# Aus UKategorie das Target extrahieren
# Wenn UKategorie = 1 oder 2 dann target 1 sonst 0
# Unfallart


crash_data["UKATEGORIE"] = crash_data["UKATEGORIE"]/2
crash_data['TARGET'] = 0 #0 leicht verletzt 1= Schwer Verletzt/getötet
crash_data.loc[crash_data["UKATEGORIE"] <= 1, ['TARGET']] = 1


def id_switcher(left_df, right_df):
    """

    :param left_df: main df
    :param right_df: merger df, col 1 = merger column; col 2 corresponding value
    :return: dataframe
    """
    on_column = right_df.columns[0]
    on_column_value = on_column + "_Value"
    left_df[on_column_value] = left_df.merge(right_df, on = on_column)['Value']
    left_df = left_df.drop([on_column],axis=1)
    return left_df


uart_df = pd.DataFrame({'UART' : [1,2,3,4,5,6,7,8,9,10], 'Value':["Zusammenstoß mit anfahrendem / anhaltendem / ruhendem Fahrzeug",
                "Zusammenstoß mit vorausfahrendem / wartendem Fahrzeug",
                "Zusammenstoß mit seitlich in gleicher Richtung fahrendem Fahrzeug",
                "Zusammenstoß mit entgegenkommendem Fahrzeug",
                "Zusammenstoß mit einbiegendem / kreuzendem Fahrzeug",
                "Zusammenstoß zwischen Fahrzeug und Fußgänger",
                "Aufprall auf Fahrbahnhindernis",
                "Abkommen von Fahrbahn nach rechts",
                "Abkommen von Fahrbahn nach links",
                "Unfall anderer Art"]})

crash_data = id_switcher(crash_data, uart_df)



# Wochentag

wochentag_df = pd.DataFrame({'UWOCHENTAG':[1,2,3,4,5,6,7],
                                 'Value':['Sonntag','Montag','Dienstag','Mittwoch','Donnerstag','Freitag','Samstag']})
crash_data = id_switcher(crash_data, wochentag_df)
    

unfalltyp_df = pd.DataFrame({'UTYP1':[1,2,3,4,5,6,7],
                                 'Value':['Fahrunfall','Abbiegeunfall','Einbiegen / Kreuzen-Unfall','Überschreitenunfall','Unfall durch ruhenden Verkehr','Unfall im Längsverkehr','sonstiger Unfall']})
crash_data = id_switcher(crash_data, unfalltyp_df)

licht_df = pd.DataFrame({'ULICHTVERH':[0,1,2],
                                 'Value':['Tageslicht','Dämmerung','Dunkelheit']})
crash_data = id_switcher(crash_data, licht_df)

str_zustand_df = pd.DataFrame({'STRZUSTAND':[0,1,2],
                                 'Value':['trocken','nass/feucht/schlüpfrig','winterglatt']})
crash_data = id_switcher(crash_data, str_zustand_df)

crash_data['DTV'] = crash_data['DTV'].fillna(crash_data['DTV'].mean())
crash_data["ANTEIL_PKW"] = crash_data["PKW"]*100/crash_data["DTV"]
crash_data["ANTEIL_PKW"] = crash_data["ANTEIL_PKW"].fillna(crash_data["ANTEIL_PKW"].mean())
crash_data["ANTEIL_LKW"] = crash_data["LKW"]*100/crash_data["DTV"]
crash_data["ANTEIL_LKW"] = crash_data["ANTEIL_LKW"].fillna(crash_data["ANTEIL_LKW"].mean())
crash_data["ANTEIL_LIEFERWAGEN"] = crash_data["LIEFERWAGEN"]*100/crash_data["DTV"]
crash_data["ANTEIL_LIEFERWAGEN"] = crash_data["ANTEIL_LIEFERWAGEN"].fillna(crash_data["ANTEIL_LKW"].mean())
crash_data["ANTEIL_LINIENBUSSE"] = crash_data["LINIENBUSSE"]*100/crash_data["DTV"]
crash_data["ANTEIL_LINIENBUSSE"] = crash_data["ANTEIL_LINIENBUSSE"].fillna(crash_data["ANTEIL_LKW"].mean())
crash_data.loc[crash_data["ANTEIL_LINIENBUSSE"] == np.inf, ['DTV']] = crash_data.loc[crash_data["ANTEIL_LINIENBUSSE"] == np.inf, ['LINIENBUSSE']]
crash_data.loc[crash_data["ANTEIL_LINIENBUSSE"] == np.inf, ['ANTEIL_LINIENBUSSE']] = 100
crash_data["ANTEIL_REISEBUSSE"] = crash_data["REISEBUSSE"]*100/crash_data["DTV"]
crash_data["ANTEIL_REISEBUSSE"] = crash_data["ANTEIL_REISEBUSSE"].fillna(crash_data["ANTEIL_REISEBUSSE"].mean())
crash_data["ANTEIL_KRAD"] = crash_data["KRAD"]*100/crash_data["DTV"]
crash_data["ANTEIL_KRAD"] = crash_data["ANTEIL_KRAD"].fillna(crash_data["ANTEIL_KRAD"].mean())
crash_data = crash_data.drop(["PKW","LKW","LIEFERWAGEN","LINIENBUSSE","REISEBUSSE","KRAD"], axis=1)
crash_data = crash_data.drop(["UKATEGORIE"], axis=1)

In [19]:
avg_ukreis = crash_data.groupby('UKREIS')['OBJECTID'].count()
count_target_ukreis = crash_data.groupby('UKREIS')['TARGET'].sum()
join_ukreis = pd.merge(avg_ukreis, count_target_ukreis, on ='UKREIS',  how='inner')
join_ukreis['UKREIS_AVG_TARGET'] = join_ukreis['TARGET']*100 / join_ukreis['OBJECTID']

avg_ugemeinde = crash_data.groupby('UGEMEINDE')['OBJECTID'].count()
count_target_ugemeinde = crash_data.groupby('UGEMEINDE')['TARGET'].sum()
join_ugemeinde = pd.merge(avg_ugemeinde, count_target_ugemeinde, on ='UGEMEINDE',  how='inner')
join_ugemeinde['UGEMEINDE_AVG_TARGET'] = join_ugemeinde['TARGET']*100 / join_ugemeinde['OBJECTID']

join_ugemeinde = join_ugemeinde.drop(["OBJECTID","TARGET"], axis=1)
join_ukreis = join_ukreis.drop(["OBJECTID","TARGET"], axis=1)

avg_dtv = crash_data.groupby('STR_NAME')['DTV'].mean()
count_crashes_by_street = crash_data.groupby('STR_NAME')['OBJECTID'].count()
count_fatal_crashes_by_street = crash_data.groupby('STR_NAME')['TARGET'].sum()
join_dtv = pd.merge(avg_dtv, count_crashes_by_street, on ='STR_NAME',  how='inner')
join_dtv = pd.merge(join_dtv, count_fatal_crashes_by_street, on ='STR_NAME',  how='inner')
join_dtv['CRASH_BY_DTV'] = (join_dtv['OBJECTID']*100 / join_dtv['DTV'])
join_dtv['CRASH_BY_DTV'] = join_dtv['CRASH_BY_DTV'].round(4)
join_dtv['FATAL_CRASH_BY_DTV'] = join_dtv['TARGET']*100 / join_dtv['DTV']
join_dtv['FATAL_CRASH_BY_DTV'] = join_dtv['FATAL_CRASH_BY_DTV'].round(4)
join_dtv = join_dtv.drop(["OBJECTID","TARGET","DTV"], axis=1)

crash_data = pd.merge(crash_data, join_ukreis, on ='UKREIS',  how='inner')
crash_data = pd.merge(crash_data, join_ugemeinde, on ='UGEMEINDE',  how='inner')
#crash_data = pd.merge(crash_data, join_dtv, on ='STR_NAME',  how='inner')
#crash_data['FATAL_CRASH_BY_DTV'] = crash_data['FATAL_CRASH_BY_DTV'].fillna(crash_data['FATAL_CRASH_BY_DTV'].mean())

In [20]:


#DTV,GESLAERM,BRW, NUTZUNG,WERT_VES,
crash_data.loc[crash_data["WERT_VES"] > 100, ['WERT_VES']] = 50
crash_data['WERT_VES'] = crash_data['WERT_VES'].fillna(50)
crash_data['UART_Value'] = crash_data['UART_Value'].fillna("Unbekannt")
crash_data['GESLAERM'] = crash_data['GESLAERM'].fillna("mittel")
crash_data['DTV'] = crash_data['DTV'].fillna(crash_data['DTV'].mean())
crash_data['DTV'] = crash_data['DTV'].fillna(0)

In [25]:
crash_data.head()

,UKREIS,UGEMEINDE,UMONAT,USTUNDE,IstRad,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstige,DTV,GESLAERM,NUTZUNG,WERT_VES,geometry,TARGET,UART_Value,UWOCHENTAG_Value,UTYP1_Value,ULICHTVERH_Value,STRZUSTAND_Value,ANTEIL_PKW,ANTEIL_LKW,ANTEIL_LIEFERWAGEN,ANTEIL_LINIENBUSSE,ANTEIL_REISEBUSSE,ANTEIL_KRAD,UKREIS_AVG_TARGET,UGEMEINDE_AVG_TARGET
0,10,10,3,11,0,1,0,0,0,0,19575.767816,mittel,W - Wohngebiet,30.0,"MULTIPOLYGON (((814997.310 5829184.012, 814996...",0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Donnerstag,Einbiegen / Kreuzen-Unfall,Tageslicht,trocken,82.408776,2.762958,2.762958,2.762958,0.400124,2.494525,21.266234,21.266234
1,10,10,1,18,0,1,0,0,0,0,6108.000000,mittel,W - Wohngebiet,30.0,"MULTIPOLYGON (((814885.204 5830057.780, 814884...",0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Donnerstag,Einbiegen / Kreuzen-Unfall,Tageslicht,trocken,81.777996,3.601834,10.216110,2.619515,0.261952,1.522593,21.266234,21.266234
2,10,10,6,7,1,1,0,0,0,0,19575.767816,niedrig - sehr niedrig,W - Wohngebiet,30.0,"MULTIPOLYGON (((814281.955 5828040.742, 814281...",1,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Donnerstag,Einbiegen / Kreuzen-Unfall,Tageslicht,trocken,82.408776,2.762958,2.762958,2.762958,0.400124,2.494525,21.266234,21.266234
3,10,10,6,13,0,1,0,0,1,0,19575.767816,mittel,W - Wohngebiet,30.0,"MULTIPOLYGON (((814215.008 5830068.156, 814214...",0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Donnerstag,Einbiegen / Kreuzen-Unfall,Tageslicht,trocken,82.408776,2.762958,2.762958,2.762958,0.400124,2.494525,21.266234,21.266234
4,10,10,5,20,0,1,0,0,0,0,3852.000000,mittel,W - Wohngebiet,30.0,"MULTIPOLYGON (((814765.428 5829893.128, 814764...",0,Zusammenstoß mit einbiegendem / kreuzendem Fah...,Donnerstag,Einbiegen / Kreuzen-Unfall,Tageslicht,trocken,81.178609,2.777778,10.202492,3.894081,0.415369,1.531672,21.266234,21.266234


In [22]:
crash_data.to_file("../01_Source/prepared_datasource.geojson")

In [23]:
crash_data = crash_data.drop(["ULAND","UREGBEZ","UJAHR","LINREFX","LINREFY","METER","EW2010","GFZ","HERKUNFT","NAME","KNAME","PLR_NAME",
                              "VBUS","VBUS_PERS","GESLAERM_PERS","GEMEINDE","BEZIRK","ANWERT","VERFAHRENSART","STICHTAG","BEITRAGSZUSTAND","LUMNUM","ELEM_NR","VRICHT_TXT","ZEIT_T","TAG_T"
                             ,"DURCH_T","DANN_T","DAT_T","LAT","LON","formatted_address","address_components","street_number","route","postal_code","sublocality","BRW"], axis=1)
crash_data = crash_data.drop(["OBJECTID","STR_NAME"], axis=1)

In [24]:
#np.all(np.isfinite(crash_data_copy))
crash_data_copy = crash_data_copy.reset_index()
np.where(crash_data_copy.values >= np.finfo(np.float64).max)

NameError: name 'crash_data_copy' is not defined

In [22]:
#crash_data.to_file("../01_Source/prepared_datasource.geojson")

In [183]:
crash_data = crash_data.drop(["OBJECTID","STR_NAME"], axis=1)


crash_data_copy = crash_data.copy()
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UART_Value"], prefix='UART_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UWOCHENTAG_Value"], prefix='UWOCHENTAG_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["UTYP1_Value"], prefix='UTYP1_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["ULICHTVERH_Value"], prefix='ULICHTVERH_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["STRZUSTAND_Value"], prefix='STRZUSTAND_Value_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["NUTZUNG"], prefix='NUTZUNG_')
crash_data_copy = pd.get_dummies(crash_data_copy, columns=["GESLAERM"], prefix='GESLAERM_')

#corr_matrix = crash_data_copy.corr()
#corr_matrix["TARGET"]
crash_data_copy = crash_data_copy.drop(["geometry"], axis=1)

In [197]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

X = crash_data_copy.drop(["TARGET"] , axis=1)
Y = crash_data_copy["TARGET"]
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=12, ratio = 1.0)
x_train_res, y_train_res = sm.fit_sample(X_train, y_train)

In [198]:

#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier()

param_grid = {
                 'n_estimators': [5, 15, 30, 100, 200],
                 'max_depth': [1, 2, 5, 7, 9, 11, 13],
                 'class_weight':["balanced"],
                 'min_samples_leaf': [2, 3, 4, 5]
             }

param_grid2 = {
                 'n_estimators': [85, 90, 95],
                 'max_depth': [3, 4, 5],
                 'min_samples_leaf': [7] #5
             }

from sklearn.model_selection import GridSearchCV

grid_clf = GridSearchCV(clf, param_grid2, cv=10, verbose=10, n_jobs=5, scoring='average_precision')
grid_clf.fit(X_train, y_train)

#Train the model using the training sets y_pred=clf.predict(X_test)
#clf.fit(X_train,y_train)

y_pred=grid_clf.predict(X_test)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:    0.5s
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed:    0.9s
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed:    1.4s
[Parallel(n_jobs=5)]: Done  22 tasks      | elapsed:    2.2s
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:    3.1s
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    3.7s
[Parallel(n_jobs=5)]: Done  51 tasks      | elapsed:    5.1s
[Parallel(n_jobs=5)]: Done  62 tasks      | elapsed:    6.3s
[Parallel(n_jobs=5)]: Done  75 tasks      | elapsed:    7.7s
[Parallel(n_jobs=5)]: Done  90 out of  90 | elapsed:    9.8s finished


In [199]:
y_pred=grid_clf.predict(X_test)

In [200]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8399267399267399


In [201]:
#grid_clf.best_params_
#{'max_depth': 1, 'n_estimators': 5}
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_pred)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.16


In [202]:
grid_clf.best_params_

{'max_depth': 5, 'min_samples_leaf': 7, 'n_estimators': 95}

In [203]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)

(2293, 0, 437, 0)

In [219]:
clf=RandomForestClassifier(random_state=42,class_weight='balanced',max_depth=3, min_samples_leaf=7, n_estimators=90)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [224]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5904761904761905


In [220]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)

(1343, 950, 168, 269)

In [226]:
#Precision
tp/(tp+fp)

0.2206726825266612

In [225]:
#Recall
tp/(tp+fn)

0.6155606407322655

In [221]:
#grid_clf.best_params_
#{'max_depth': 1, 'n_estimators': 5}
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_pred)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

Average precision-recall score: 0.20


In [24]:
y_dummy = []

In [25]:
for element in y_pred:
    y_dummy.append(0)

In [26]:
tn, fp, fn, tp = confusion_matrix(y_test, y_dummy).ravel()
(tn, fp, fn, tp)

(2285, 0, 445, 0)

In [128]:
feature_imp = pd.Series(clf.feature_importances_, index= X_train.columns).sort_values(ascending=False)

In [129]:
feature_imp

IstFuss                                                                          0.203049
IstKrad                                                                          0.081867
UGEMEINDE                                                                        0.074144
ANTEIL_KRAD                                                                      0.069947
USTUNDE                                                                          0.067844
UKREIS                                                                           0.067794
IstPKW                                                                           0.061962
ANTEIL_REISEBUSSE                                                                0.054484
ANTEIL_LIEFERWAGEN                                                               0.037665
IstRad                                                                           0.034589
DTV                                                                              0.032861
ANTEIL_LKW

In [18]:
crash_data_copy["TARGET"].value_counts()

0    11433
1     2216
Name: TARGET, dtype: int64

In [20]:
(11433-2216)/11433

0.806175107145981